# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 11:31AM,257250,19,ADV80011703,"AdvaGen Pharma, Ltd",Released
1,Feb 13 2023 11:31AM,257250,19,ADV80011704,"AdvaGen Pharma, Ltd",Released
2,Feb 13 2023 11:31AM,257250,19,ADV80011705,"AdvaGen Pharma, Ltd",Released
3,Feb 13 2023 11:31AM,257250,19,ADV80011706,"AdvaGen Pharma, Ltd",Released
4,Feb 13 2023 11:31AM,257250,19,ADV80011707,"AdvaGen Pharma, Ltd",Released
5,Feb 13 2023 11:31AM,257250,19,ADV80011712,"AdvaGen Pharma, Ltd",Released
6,Feb 13 2023 11:31AM,257250,19,ADV80011713,"AdvaGen Pharma, Ltd",Released
7,Feb 13 2023 11:31AM,257250,19,ADV80011714,"AdvaGen Pharma, Ltd",Released
8,Feb 13 2023 11:31AM,257250,19,ADV80011715,"AdvaGen Pharma, Ltd",Released
9,Feb 13 2023 11:31AM,257250,19,ADV80011716,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,257242,Released,3
41,257245,Released,2
42,257247,Released,1
43,257249,Released,4
44,257250,Released,26


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257242,NaN,NaN,3.0
257245,NaN,NaN,2.0
257247,NaN,NaN,1.0
257249,NaN,NaN,4.0
257250,NaN,NaN,26.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,0.0,2.0,1.0
257194,35.0,5.0,1.0
257195,0.0,1.0,41.0
257198,0.0,1.0,0.0
257199,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,0,2,1
257194,35,5,1
257195,0,1,41
257198,0,1,0
257199,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,0,2,1
1,257194,35,5,1
2,257195,0,1,41
3,257198,0,1,0
4,257199,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,,2,1
1,257194,35,5,1
2,257195,,1,41
3,257198,,1,
4,257199,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd"
26,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc."
30,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc."
31,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc"
33,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc."
36,Feb 13 2023 11:03AM,257241,10,Eywa Pharma Inc.
37,Feb 13 2023 10:54AM,257240,19,"GCH Granules USA, Inc."
40,Feb 13 2023 10:48AM,257238,10,Eywa Pharma Inc.
51,Feb 13 2023 10:41AM,257231,10,ISDIN Corporation
114,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd",,,26
1,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc.",,,4
2,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc.",,,1
3,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc",,,2
4,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc.",,,3
5,Feb 13 2023 11:03AM,257241,10,Eywa Pharma Inc.,,,1
6,Feb 13 2023 10:54AM,257240,19,"GCH Granules USA, Inc.",,,3
7,Feb 13 2023 10:48AM,257238,10,Eywa Pharma Inc.,,,11
8,Feb 13 2023 10:41AM,257231,10,ISDIN Corporation,,,63
9,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,,,46


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd",26,,
1,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc.",4,,
2,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc.",1,,
3,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc",2,,
4,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc.",3,,
5,Feb 13 2023 11:03AM,257241,10,Eywa Pharma Inc.,1,,
6,Feb 13 2023 10:54AM,257240,19,"GCH Granules USA, Inc.",3,,
7,Feb 13 2023 10:48AM,257238,10,Eywa Pharma Inc.,11,,
8,Feb 13 2023 10:41AM,257231,10,ISDIN Corporation,63,,
9,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,46,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd",26,,
1,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc.",4,,
2,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc.",1,,
3,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc",2,,
4,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd",26.0,NaN,NaN
1,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc.",4.0,NaN,NaN
2,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc.",1.0,NaN,NaN
3,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc",2.0,NaN,NaN
4,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 11:31AM,257250,19,"AdvaGen Pharma, Ltd",26.0,0.0,0.0
1,Feb 13 2023 11:29AM,257249,10,"Citieffe, Inc.",4.0,0.0,0.0
2,Feb 13 2023 11:27AM,257247,19,"Methapharm, Inc.",1.0,0.0,0.0
3,Feb 13 2023 11:24AM,257245,12,"Uniderm USA, Inc",2.0,0.0,0.0
4,Feb 13 2023 11:07AM,257242,10,"Methapharm, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6173293,436.0,2.0,0.0
12,771668,5.0,0.0,0.0
15,771677,67.0,36.0,0.0
19,1543324,72.0,7.0,35.0
21,514441,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6173293,436.0,2.0,0.0
1,12,771668,5.0,0.0,0.0
2,15,771677,67.0,36.0,0.0
3,19,1543324,72.0,7.0,35.0
4,21,514441,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,436.0,2.0,0.0
1,12,5.0,0.0,0.0
2,15,67.0,36.0,0.0
3,19,72.0,7.0,35.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,436.0
1,12,Released,5.0
2,15,Released,67.0
3,19,Released,72.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,0.0,0.0,0.0,35.0,0.0
Executing,2.0,0.0,36.0,7.0,0.0
Released,436.0,5.0,67.0,72.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,0.0,0.0,0.0,35.0,0.0
1,Executing,2.0,0.0,36.0,7.0,0.0
2,Released,436.0,5.0,67.0,72.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,0.0,0.0,0.0,35.0,0.0
1,Executing,2.0,0.0,36.0,7.0,0.0
2,Released,436.0,5.0,67.0,72.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()